# Local MPChecker

We can use `jorbit` to mimic the MPC's "MPChecker" service. This is a service that, when given a sky coordinate and time, will return a list of all known objects that are in the field of view of a given telescope. This is useful for identifying observed objects in a given field, or for checking for contamination of another science target by solar system objects.

The `jorbit` version is not guaranteed to successfully flag every object: there are a number of simplifications where a particle could fall through the cracks. However, it offers a chance for a rapid first pass, and the the ability to run all computations locally (after downloading some cached data, which is done automatically using Astropy).

In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

from jorbit.mpchecker import mpchecker

For the fastest but crudest check, we can run:

In [2]:
mpchecker(
    coordinate=SkyCoord(ra=0 * u.deg, dec=0 * u.deg),
    time=Time("2025-01-01"),
    radius=10 * u.arcmin,
    extra_precision=False,
)

Unpacked Name,separation,ra,dec,Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,last obs
,arcsec,deg,deg,,,,,,,,,,,,,,,,,,,,,,
str19,float64,float64,float64,str7,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,str1,str9,float64,float64,str9,float64,str3,str3,str8,str4,str8
2011 UE381,248.51519282838578,359.9353047057922,0.024081858375017095,K11Uc1E,18.09,0.15,K2555,65.96611,125.73907,204.52407,13.88006,0.1853965,0.22360622,2.6883149,0,MPO893282,97.0,7.0,2010-2024,0.47,M-v,3Ek,MPCLINUX,0000,20241122
2016 BK84,270.42781198032526,359.9281601096067,-0.021951538035610858,K16B84K,17.32,0.15,K2555,255.37314,140.798,15.97159,10.97567,0.172949,0.17929154,3.1147953,0,E2024-TN4,84.0,7.0,2010-2024,0.42,M-v,3Ek,MPCLINUX,0000,20241010
(243996) 2001 RP149,299.5286772173218,0.034387214576621715,-0.07576385186187751,O3996,17.8,0.15,K2555,31.43209,190.53068,205.56806,1.35343,0.0988646,0.30001667,2.2098992,0,MPO889888,396.0,15.0,2000-2024,0.89,M-v,3Ek,MPCLINUX,0000,20241109
(328594) 2009 SU69,328.03233241021195,0.040290737741438314,0.08172838417960507,W8594,16.75,0.15,K2555,178.62152,23.55454,199.3523,5.64506,0.0341326,0.19680507,2.9271515,0,E2024-TI9,223.0,13.0,1994-2024,0.99,M-v,3Ek,MPCLINUX,0000,20241006
(192469) 1998 FF48,362.6266227767734,0.044893084395516265,-0.09017243739974345,J2469,15.66,0.15,K2555,207.7489,2.37492,199.18862,8.44388,0.1112137,0.24058623,2.5602883,0,E2025-BC9,880.0,18.0,1994-2025,0.79,M-v,3Ek,MPCLINUX,0000,20250124
2020 SE100,365.93508442844364,0.005957379884145937,-0.10147391078813246,K20SA0E,18.59,0.15,K2555,37.22087,170.49025,203.82952,2.42053,0.1632445,0.23272611,2.6176162,0,E2024-TA7,49.0,4.0,2003-2024,0.38,M-v,3Ek,MPCLINUX,0000,20241004
(572408) 2008 GD168,394.4635076095755,359.890512021332,-0.004320642764798248,v2408,14.52,0.15,K2555,83.14212,274.6067,10.43473,5.36024,0.1147046,0.08322819,5.1954409,0,E2024-U66,109.0,9.0,2008-2024,0.98,M-v,3Ek,MPCLINUX,0009,20241023
(167818) 2005 CH8,479.06228574214134,0.13284005307666424,-0.007868018793499496,G7818,16.15,0.15,K2555,155.50455,219.06687,21.24485,2.00217,0.1476283,0.22873664,2.6479649,0,E2025-B02,650.0,19.0,1971-2025,0.86,M-v,3Ek,MPCLINUX,0000,20250116


This should take less than a second (if the appropriate files are downloaded), but makes a few simplifications. First, the true underlying positions of the objects here were integrated using only Newtonian gravity and perturbations from the planets and major asteroids. By neglecting effects like GR, we introduce a ~arsec level error (usually much smaller, but we'll be conservative). Second, when converting these positions into sky coordinates, we've assumed that the observer is located at the geocenter. Depending on the observer's true location, this can introduce $>$arcsec level errors, especially if the object is nearby and the observer is actually a spacecraft like TESS or JWST.

So, for a more accurate check, we can run:

In [3]:
mpchecker(
    coordinate=SkyCoord(ra=0 * u.deg, dec=0 * u.deg),
    time=Time("2025-01-01"),
    radius=10 * u.arcmin,
    extra_precision=True,
    observer="palomar",
)

Unpacked Name,separation,ra,dec,est. Vmag,Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,last obs
,arcsec,deg,deg,mag,,,,,,,,,,,,,,,,,,,,,,
str19,float64,float64,float64,float64[1],str7,float64,float64,str5,float64,float64,float64,float64,float64,float64,float64,str1,str9,float64,float64,str9,float64,str3,str3,str8,str4,str8
2011 UE381,246.9656208508839,359.9355513614736,0.023506327753215366,22.883896504489698,K11Uc1E,18.09,0.15,K2555,65.96611,125.73907,204.52407,13.88006,0.1853965,0.22360622,2.6883149,0,MPO893282,97.0,7.0,2010-2024,0.47,M-v,3Ek,MPCLINUX,0000,20241122
2016 BK84,270.2729766259403,359.92832146895455,-0.022328641316682874,23.638583994023666,K16B84K,17.32,0.15,K2555,255.37314,140.798,15.97159,10.97567,0.172949,0.17929154,3.1147953,0,E2024-TN4,84.0,7.0,2010-2024,0.42,M-v,3Ek,MPCLINUX,0000,20241010
(243996) 2001 RP149,302.24899998086346,0.03468578623390513,-0.07645817182783533,21.977731393251847,O3996,17.8,0.15,K2555,31.43209,190.53068,205.56806,1.35343,0.0988646,0.30001667,2.2098992,0,MPO889888,396.0,15.0,2000-2024,0.89,M-v,3Ek,MPCLINUX,0000,20241109
(328594) 2009 SU69,326.9228844814677,0.04047974093849042,0.08129080574573799,22.51069623801466,W8594,16.75,0.15,K2555,178.62152,23.55454,199.3523,5.64506,0.0341326,0.19680507,2.9271515,0,E2024-TI9,223.0,13.0,1994-2024,0.99,M-v,3Ek,MPCLINUX,0000,20241006
(192469) 1998 FF48,364.4560548639444,0.045094190751591455,-0.09063998230313255,21.195954694999276,J2469,15.66,0.15,K2555,207.7489,2.37492,199.18862,8.44388,0.1112137,0.24058623,2.5602883,0,E2025-BC9,880.0,18.0,1994-2025,0.79,M-v,3Ek,MPCLINUX,0000,20250124
2020 SE100,368.2213767355501,0.006223493738215968,-0.10209420491061076,23.140906345759323,K20SA0E,18.59,0.15,K2555,37.22087,170.49025,203.82952,2.42053,0.1632445,0.23272611,2.6176162,0,E2024-TA7,49.0,4.0,2003-2024,0.38,M-v,3Ek,MPCLINUX,0000,20241004
(572408) 2008 GD168,394.10565757630127,359.89062197241026,-0.004578059489983028,22.300205354650778,v2408,14.52,0.15,K2555,83.14212,274.6067,10.43473,5.36024,0.1147046,0.08322819,5.1954409,0,E2024-U66,109.0,9.0,2008-2024,0.98,M-v,3Ek,MPCLINUX,0009,20241023
(167818) 2005 CH8,479.87000405465443,0.13303713743147835,-0.008322856537808272,21.779069200118897,G7818,16.15,0.15,K2555,155.50455,219.06687,21.24485,2.00217,0.1476283,0.22873664,2.6479649,0,E2025-B02,650.0,19.0,1971-2025,0.86,M-v,3Ek,MPCLINUX,0000,20250116


This will take longer, but should remedy the above issues. Under the hood, this first runs the approximate version to flag which objects were in the field of view, then runs a fresh N-body integration for each on that fully accounts for the observer's location and relativistic effects. The final coordinates should agree to ~1 mas with horizons for times 2010-2030. 

As a plus, we now also can add an extra column, "est. Vmag": since we now know the true distance to the object and its phase angle, we might as well convert its absolute magnitude into an estimated V-band magnitude.

Note though that if any of our above assumptions were egregiously wrong and a particle wasn't flagged, we'll still be out of luck. Also note that this now requires a connection to the internet to request the locations of the observer from Horizons (though the initial positions of the objects are still cached locally).